In [1]:
## IMPORT ALL LIBRAIES...

import os
import copy
import pandas as pd 
import numpy as np 
import torch
import albumentations as A
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import torch.nn as nn 
from albumentations.pytorch.transforms import ToTensorV2
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision.transforms import Compose
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader 
import torchvision.models as models
from ImageProcess import SplitImages, checkingsplitsame
from ImageClfModeling import set_parameter_requires_grad, train_model


#--------------------------------------------IMAGE PATH---------------------------------------------

# image path
imagepath = "Image/Images/"


#--------------------------------------------INITIAL GLOBAL TRANSFORMATION---------------------------------------------
# global transformation into tensor form 
global_transform = Compose([transforms.ToTensor()])
database = ImageFolder(root=imagepath, transform=global_transform)


#--------------------------------------------CLASS LABELS TO INDEX---------------------------------------------

# class names from index to class
class_names = [x for x in database.classes]
class_name_dict = {}
for ix, val in enumerate(class_names):
    class_name_dict[ix] = val
    
    
#--------------------------------------------RANDOMIZED IMAGE SPLIT---------------------------------------------

# random images split 
valid_div, test_div, randomseed = 20, 10, 10
valid_sz = len(database)//valid_div
test_sz = len(database)//test_div
train_ds, valid_ds, test_ds = SplitImages(database, valid_sz, test_sz, randomseed)


#--------------------------------------------SPLIT IMAGES TEST---------------------------------------------
print("TESTING SPLIT IMAGES")
print()
# testing indices for test data with train data 
checkingsplitsame(test_ds, train_ds, "Train")

# testing indices for test data with validation data 
checkingsplitsame(valid_ds, test_ds, "Valid")

# testing indices for validation data with train data 
checkingsplitsame(valid_ds, train_ds, "Test")


#--------------------------------------------DATA AUGMENTATION---------------------------------------------

# train transformation 
print()
print("AUGMENTING TRAINING IMAGES")
print()
train_transform = A.Compose([A.HorizontalFlip(p=0.5), 
                            A.ImageCompression(quality_lower=99, quality_upper=100),
                            A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10,
                                              border_mode=0, p=0.7),
                            A.Resize(224, 224),
                            A.Cutout(max_h_size=int(224*0.4), max_w_size=int(224*0.4), 
                                    num_holes=1, p=0.65),
                            ToTensorV2()])

traindata_trans = []

for img, lb in tqdm(train_ds):
    im = train_transform(image=img.permute(1,2,0).numpy())
    traindata_trans.append((im["image"], lb))
    
    
# validation transformation 
print()
print("AUGMENTING VALIDATION IMAGES")
print()
valid_transform = A.Compose([A.Resize(224, 244),
                            ToTensorV2()])

validdata_trans = []

for img, lb in tqdm(valid_ds):
    im = valid_transform(image = img.permute(1,2,0).numpy())
    validdata_trans.append((im["image"], lb))


# test data transformation 
print()
print("AUGMENTING TEST IMAGES")
print()
test_transform = A.Compose([A.Resize(224, 244),
                            ToTensorV2()])

testdata_trans = []

for img, lb in tqdm(test_ds):
    im = test_transform(image = img.permute(1,2,0).numpy())
    testdata_trans.append((im["image"], lb))
    
    
#--------------------------------------------DATA LOADERS---------------------------------------------  


# batch size for train and validation data 
batch_size_ = 64

# batch size for test data (we want to test all the data at one go)
batch_test = len(test_ds)

# Iterative Data loader
train_dl = DataLoader(traindata_trans, batch_size=batch_size_, shuffle=True, num_workers=2)
valid_dl = DataLoader(validdata_trans, batch_size=batch_size_, shuffle=True, num_workers=2)
test_dl = DataLoader(testdata_trans, batch_size=batch_test, shuffle=False, num_workers=2)

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


TESTING SPLIT IMAGES

Train: Datset is not Matching
Valid: Datset is not Matching
Test: Datset is not Matching

AUGMENTING TRAINING IMAGES



/home/ubuntu/.local/lib/python3.8/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


  0%|          | 0/561 [00:00<?, ?it/s]

/home/ubuntu/.local/lib/python3.8/site-packages/albumentations/augmentations/functional.py:485: UserWarning: Image compression augmentation is most effective with uint8 inputs, float32 is used as input.
  warn(
/usr/lib/python3/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))



AUGMENTING VALIDATION IMAGES



  0%|          | 0/33 [00:00<?, ?it/s]


AUGMENTING TEST IMAGES



  0%|          | 0/66 [00:00<?, ?it/s]

In [2]:
#-------------------------------MODEL---------------------------------------


# Initialising the pre trained model
'''def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = models.resnet50(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc =  nn.Sequential(nn.Dropout(0.1),nn.Linear(num_ftrs,num_classes))
    return model_ft'''


def initialize_model(num_classes, feature_extract, use_pretrained=True):
    model_ft = models.alexnet(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    model_last_inp = model_ft.classifier[6].in_features
    model_ft.classifier[6] = nn.Linear(model_last_inp, num_classes)
    return model_ft

In [3]:
#--------------------------------------INITIALIZING PRE TRAINED MODEL-----------------------------------


num_classes = len(class_name_dict)
feature_extract = False

# pre trained model
model_ft = initialize_model(num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

/usr/lib/python3/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/usr/lib/python3/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [4]:
#--------------------------------SENDING MODEL TO GPU-------------------------------------


# Send the model to GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.benchmark = True
print(f'using {device} device')

model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")

for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

using cuda device
Params to learn:
	 features.0.weight
	 features.0.bias
	 features.3.weight
	 features.3.bias
	 features.6.weight
	 features.6.bias
	 features.8.weight
	 features.8.bias
	 features.10.weight
	 features.10.bias
	 classifier.1.weight
	 classifier.1.bias
	 classifier.4.weight
	 classifier.4.bias
	 classifier.6.weight
	 classifier.6.bias


In [5]:
#--------------------------------MODEL PARAMETERS---------------------------------------------


# Epochs
epochs_ = 50

# learning rate
lr= 1e-3

# momentum
moment = 0.9

# Optimizer
optimizer_ft = torch.optim.SGD(params_to_update, lr=lr, momentum=moment)

# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

In [6]:
#----------------------------------- TRAIN & VALIDATE--------------------------------------

#dataloaders
dataloaders_dict ={}
dataloaders_dict['train']= train_dl
dataloaders_dict['valid'] = valid_dl

# Train and evaluate
model_ft, epoch_loss_train, epoch_acc_train, epoch_loss_valid, epoch_acc_valid  = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=epochs_)

Epoch: 0| train loss: 136.30084228515625| valid loss: 50.32771301269531| train acc: 0.504960298538208| valid acc: 0.9090909361839294
Epoch: 1| train loss: 77.51609802246094| valid loss: 30.032184600830078| train acc: 0.9017148613929749| valid acc: 0.8787878751754761
Epoch: 2| train loss: 54.11399841308594| valid loss: 28.930511474609375| train acc: 0.9284297227859497| valid acc: 0.9393939971923828
Epoch: 3| train loss: 40.85371780395508| valid loss: 25.552658081054688| train acc: 0.959537923336029| valid acc: 0.9393939971923828
Epoch: 4| train loss: 31.029998779296875| valid loss: 31.859148025512695| train acc: 0.975162923336029| valid acc: 0.9393939971923828
Epoch: 5| train loss: 22.94483184814453| valid loss: 24.445480346679688| train acc: 0.9965277910232544| valid acc: 0.9393939971923828
Epoch: 6| train loss: 15.99258804321289| valid loss: 20.68382453918457| train acc: 0.9982638955116272| valid acc: 0.9393939971923828
Epoch: 7| train loss: 13.619832038879395| valid loss: 33.42455291

In [7]:
#-----------------------------TEST IMAGES-----------------------

# input data 
for inputs, lbs in test_dl:
    if len(inputs)==batch_test:
        inputs = inputs.to(device) 
        model_ft.eval()
        with torch.no_grad():
            outputs_ = model_ft(inputs)

In [8]:
from ImageClfModeling import topk_accuracy

topk_accuracy(outputs_, lbs.to(device))

tensor([0.9848], device='cuda:0')

In [11]:
#--------------------------------SAVE MODEL---------------------------

# save the labels

df_label = pd.DataFrame()
df_label["Labels"] = list(class_name_dict.keys())
df_label["Label_name"] = list(class_name_dict.values())

df_label.to_csv("scence_name.csv", index=False)

torch.save(model_ft.state_dict(), "AlexNet_SceneImageClassificationWeight.pt")